**PROJECT TITLE:** AI Resume & Cover Letter Generator


---

**DESCRIPTION:**

It's frustating to manually write your resume or cover letter. Now try this project below where you just have to provide the key details like your NAME, JOB ROLE, SKILLS, etc. and rest back and let this project do rest of the work for you. Not only it will generate your resume and cover letter but also automatically download the file in form of PDF in your system which you can edit whenever you want.


---
***Challenges Faced: ***
1. Handling PDF formatting from raw AI output.
2. Creating flexible prompts that return high quality text.
3. Managing various types of user input.
4. Ensuring that the output resumes have a professional structure.


---



**SOLUTION:**



1. Simple user input system (Nothing COMPLEX!)
2. AI content generation
3. Downloadable PDF output
4. Error handling for missing input & API issues


---

**TOOLS USED:**

1. **Platform** -> Google Colab
2. **Language** -> Python
3. **AI Model** -> Google Gemini (Gemini 2.5 Flash)
4. **PDF Generation** -> ReportLab library


---
**HOW THINGS WORKOUT:**

1. **User Input** -> Takes input from user [USERS PERSONAL API KEY]
2. **Prompt Creation** -> Based on the user's input a detailed prompt is created
3. **AI Text Generation** -> Using the Google Gemini API & API key. The prompt is sent to the Gemini model.  
4. **PDF Generation** -> PDF is generated for Resume & Cover Letter
5. **Download Files** -> Finally! PDF files are automatically
downloaded to the user’s system.

---
Let's check it out!







In [1]:
# Install necessary libraries
!pip install google-generativeai
!pip install reportlab

# Import required modules
import os
from IPython.display import display, Markdown
from google.colab import files
import google.generativeai as genai

# Import ReportLab modules for PDF generation
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.lib.enums import TA_LEFT, TA_CENTER, TA_RIGHT, TA_JUSTIFY
from reportlab.lib.colors import black, blue, darkgreen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.9 MB/s eta 0:00:00


# Task
Add a Gradio interface to the existing code that allows users to input their resume details, generate a resume, and download it as a PDF.

## Define gradio inputs

### Subtask:
Determine the Gradio components needed to collect all the required resume information from the user (full name, email, phone number, work experience details, education details, skills, and resume template choice).


**Reasoning**:
Identify the necessary Gradio components for collecting user input for the resume, considering how to handle the list inputs for work experience and education within the Gradio framework.



In [2]:
import gradio as gr

# Define Gradio components for user input
name_input = gr.Textbox(label="Full Name")
email_input = gr.Textbox(label="Email Address")
phone_input = gr.Textbox(label="Phone Number")

# For work experience, we can use a Dataframe component to allow users to add multiple entries.
# We define the headers that match the keys in the work_experience dictionary.
experience_input = gr.Dataframe(
    headers=["job_title", "company_name", "location", "start_date", "end_date"],
    label="Work Experience",
    row_count=(0, "dynamic"),  # Allow dynamic addition of rows
    col_count=(5, "fixed"),
    interactive=True
)

# Similarly, for education, use a Dataframe.
education_input = gr.Dataframe(
    headers=["degree", "institution", "graduation_date"],
    label="Education",
    row_count=(0, "dynamic"),
    col_count=(3, "fixed"),
    interactive=True
)

skills_input_text = gr.Textbox(label="Skills (comma-separated)")

# Resume template selection using a Dropdown
resume_template_choice_input = gr.Dropdown(
    choices=["Template A", "Template B", "Template C"],
    label="Choose Resume Template"
)

## Create a wrapper function for gradio

### Subtask:
Write a Python function that will be called by the Gradio interface. This function will take the inputs from the Gradio components, format them into the `user_information` dictionary structure expected by `generate_resume`, call `generate_resume`, save the output as a PDF using `save_as_pdf`, and return the path to the generated PDF file.


**Reasoning**:
Define the `generate_resume_interface` function to process Gradio inputs, call the existing resume generation and saving functions, and return the PDF file path.



In [3]:
def generate_resume_interface(name, email, phone, experience_df, education_df, skills_text, resume_template_choice):
    """
    Processes Gradio inputs, generates a resume PDF, and returns the file path.
    """
    # Convert experience DataFrame to list of dictionaries
    work_experience = []
    if not experience_df.empty:
        for index, row in experience_df.iterrows():
            # Handle potential NaN values in Dataframe rows
            cleaned_row = {k: ('' if pd.isna(v) else str(v)) for k, v in row.items()}
            work_experience.append(cleaned_row)

    # Convert education DataFrame to list of dictionaries
    education = []
    if not education_df.empty:
        for index, row in education_df.iterrows():
             # Handle potential NaN values in Dataframe rows
            cleaned_row = {k: ('' if pd.isna(v) else str(v)) for k, v in row.items()}
            education.append(cleaned_row)

    # Convert skills string to list
    skills = [skill.strip() for skill in skills_text.split(',') if skill.strip()]

    # Create user_information dictionary
    user_information = {
        'full_name': name,
        'email': email,
        'phone_number': phone,
        'work_experience': work_experience,
        'education': education,
        'skills': skills
    }

    # Generate the resume string
    generated_resume = generate_resume(user_information, resume_template_choice)

    # Define filename and save as PDF
    filename = "resume.pdf"
    save_as_pdf(generated_resume, filename)

    return filename

## Create the gradio interface

### Subtask:
Instantiate a `gr.Interface` object, linking the input components to the wrapper function and defining the output component (which should be a `gr.File` component to allow downloading the PDF).


**Reasoning**:
Instantiate a gr.Interface object with the defined inputs, function, and output component.



In [4]:
# Create the Gradio Interface
resume_interface = gr.Interface(
    fn=generate_resume_interface,
    inputs=[
        name_input,
        email_input,
        phone_input,
        experience_input,
        education_input,
        skills_input_text,
        resume_template_choice_input
    ],
    outputs=gr.File(label="Download Resume PDF"),
    title="AI Resume Generator",
    description="Enter your details to generate and download your resume as a PDF."
)

# The interface can be launched with resume_interface.launch()
# We will not launch it automatically in this cell.

## Launch the gradio interface

### Subtask:
Launch the Gradio web server to make the AI Resume Generator accessible through a web interface.


**Reasoning**:
Launch the Gradio interface to make it accessible via a web link.



In [5]:
# Launch the Gradio interface with sharing enabled
resume_interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://43dbb405ebdac7b187.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Challenges:

## Challenges and Solutions
1.✅Handling API Failures
Challenge: Invalid API keys or model names caused the app to crash.

Solution:
Added try-except blocks to catch API errors and display them in the UI instead of crashing. The code was updated to use a stable model name (gemini-1.5-flash-latest) to ensure reliability.


2.✅Simplifying User Input
Challenge: The interface for adding a resume (upload vs. paste) was cluttered.

Solution:
Used a Gradio radio button to dynamically show only the relevant input field (either the file uploader or the text area), creating a cleaner user experience.


3.✅Clean File Downloads
Challenge: Offering both PDF and TXT downloads resulted in large, confusing UI buttons.

Solution:

Switched to a single gr.File(file_count="multiple") component, which neatly lists all downloadable files provided by the backend, fixing the UI.


4.✅Parsing Inconsistent PDF Resume Formats
Challenge: Resumes uploaded as PDFs have varied layouts (e.g., multi-column, tables). PyPDF2 can fail to extract text in a logical order from complex formats, providing jumbled content to the AI and resulting in a poor-quality cover letter.

Solution:

The primary solution was to offer a "Paste Text" option as a reliable alternative. This allows users to manually copy and paste their resume content, bypassing any PDF parsing issues and guaranteeing the AI receives clean, properly ordered text.

5.✅Translating AI Formatting to PDF Output
Challenge: The Gemini model often generates text with markdown for formatting (like **Headings** or * Bullet points), but the fpdf2 library doesn't understand markdown and would just print it as plain text.

Solution:

A simple post-processing function was created to parse the AI's text output. This function iterates through the response line-by-line, detects markdown cues, and applies the corresponding fpdf2 formatting command (e.g., setting the font to bold for headings or adding a bullet symbol for list items) before writing to the PDF.

## Summary:

### Data Analysis Key Findings

*   Gradio input components were defined using `gr.Textbox` for single text fields, `gr.Dataframe` for dynamic lists of work experience and education entries, and `gr.Dropdown` for selecting a resume template.
*   A Python function `generate_resume_interface` was created to act as a wrapper for the Gradio interface, handling the conversion of Gradio inputs (including pandas DataFrames for lists) into the format required by the resume generation logic.
*   The `generate_resume_interface` function includes error handling for potential `NaN` values in the DataFrames and converts the comma-separated skills string into a list.
*   A `gr.Interface` object named `resume_interface` was successfully instantiated, linking the defined input components to the `generate_resume_interface` function and setting the output as a `gr.File` component for PDF download.
*   The Gradio interface was launched successfully, generating a public URL for web access to the AI Resume Generator.

### Insights or Next Steps

*   Consider adding validation to the Gradio inputs (e.g., email format, date formats) to ensure data quality before generating the resume.
*   Enhance the `gr.Dataframe` inputs with more specific column types or validation rules if possible, to guide user input for experience and education details.
